In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, MultiHeadAttention, LayerNormalization, Add
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from collections import Counter
import nbformat

2024-12-05 19:11:06.308576: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-05 19:11:06.928632: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733425867.146746    8143 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733425867.211928    8143 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-05 19:11:07.799662: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
%run '/home/ec2-user/kaggle/rudolph/dataPrep.ipynb'

Story text extracted successfully:
Story text extracted successfully:
Story text extracted successfully:
Story text extracted successfully:
Story text extracted successfully:
Story text extracted successfully:
Story text extracted successfully:
Story text extracted successfully:
Story text extracted successfully:
Story text extracted successfully:
Story text extracted successfully:
Story text extracted successfully:
Story text extracted successfully:
Story text extracted successfully:
Story text extracted successfully:
Story text extracted successfully:
Story text extracted successfully:
Story text extracted successfully:
Story text extracted successfully:
Story text extracted successfully:
Story text extracted successfully:
Story text extracted successfully:
Story text extracted successfully:
A Boy was given permission to put his hand into a pitcher to get some filberts. But he took such a great fistful that he could not draw his hand out again. There he stood, unwilling to give up a 

In [3]:
print(stories[0][0])
print(stories[0][1])

A Boy was given permission to put his hand into a pitcher to get some filberts. But he took such a great fistful that he could not draw his hand out again. There he stood, unwilling to give up a single filbert and yet unable to get them all out at once. Vexed and disappointed he began to cry.
"My boy," said his mother, "be satisfied with half the nuts you have taken and you will easily get your hand out. Then perhaps you may have some more filberts some other time."
Do not attempt too much at once.



Return to theAesoplibrary , or . . . Read the next short story;The Boy And The Nettle
Or read more short stories for kids in ourChildren's Library
into other to easily out But his theAesoplibrary , Library them great out have There at mother, stories The took you was with hand to may his Boy Nettle your not at boy," and not "My a taken stood, once. said up next filbert once. or began and perhaps fistful Vexed you that too single . more half put he ourChildren's filberts satisfied much . D

In [3]:
# Sample dataset: Pairs of original and scrambled sentences 
"""
sentences = [
    ("I love programming", "programming I love"),
    ("Keras makes deep learning easy", "makes easy Keras deep learning"),
    ("The cat sat on the mat", "on the mat The cat sat"),
    ("Hello world", "world Hello"),
    ("Pine Cone and Pepper Pot ran after her. Jeremy Mouse only stuck his head out of the snow cave. He didn’t want Jack Frost to nip his toes and certainly not his tail! Jack Frost! Jack Frost! cried Tiptoes. We’ve come to pay a visit. Jack Frost landed next to them. He was pale, icy blue, and his arms and fingers were long and glistening. His eyes were sharp, and his wings crackled like breaking ice whenever he moved. He looked so fierce that Pine Cone and Pepper Pot were afraid to open their mouths. It became so chilly they wrapped their beards round their necks and pulled their hats over their ears.", "snow he beards only fingers so out their We’ve of and came his sharp, they Cone her. afraid glistening. he Jack long, their It Jack He open his Pot! was to Pot and hats nip after not He necks the ears. Jeremy over Jack next icy Mouse round moved. He tail! was pale, the pay their icy pine wrapped their didn’t heads ice fierce glistening. mouths. breaking over pay Jack next like arms his moved. and certainly! only out was the look Tiptoes over not mouths. and he his to Pepper afraid cave. him Jeremy stuck crackled came blue, pulled ran his round with tail! He them. Mouse after didn’t nip fingers wings their not tail! mouse Pot afraid stuck arms of toes icy Cone was them pay. Pine wings came Pepper their He fierce long his wings Cone hats icy his over and next certainly Jack after ears. glistening. pay tiptoes! like pine ran was icy cave! certainly moved the."),
    ("Machine learning is fascinating", "fascinating Machine is learning"),
]
"""
sentences = stories

In [ ]:
#print(sentences[0])
#print(stories[0])


('I love programming', 'programming I love')
('A Boy was given permission to put his hand into a pitcher to get some filberts. But he took such a great fistful that he could not draw his hand out again. There he stood, unwilling to give up a single filbert and yet unable to get them all out at once. Vexed and disappointed he began to cry.\n"My boy," said his mother, "be satisfied with half the nuts you have taken and you will easily get your hand out. Then perhaps you may have some more filberts some other time."\nDo not attempt too much at once.\n\n\n\nReturn to theAesoplibrary , or . . . Read the next short story;The Boy And The Nettle\nOr read more short stories for kids in ourChildren\'s Library', 'into other to easily out But his theAesoplibrary , Library them great out have There at mother, stories The took you was with hand to may his Boy Nettle your not at boy," and not "My a taken stood, once. said up next filbert once. or began and perhaps fistful Vexed you that too single . 

In [4]:
# New Add <start> and <end> tokens to the original sentences because it seems to break the algorithm without them
original_sentences = [f"<start> {s[0]} <end>" for s in sentences]
scrambled_sentences = [s[1] for s in sentences]

# Tokenize and preprocess sentences
tokenizer = Tokenizer(filters="")
tokenizer.fit_on_texts(original_sentences + scrambled_sentences)
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1

max_seq_length = max(
    max(len(seq.split()) for seq in scrambled_sentences),
    max(len(seq.split()) for seq in original_sentences),
)
#max_seq_length = 46914 #inputted to avoid resource drain of run data prep file.
# Convert sentences to sequences and pad
input_sequences = pad_sequences(tokenizer.texts_to_sequences(scrambled_sentences), maxlen=max_seq_length, padding="post")
target_sequences = pad_sequences(tokenizer.texts_to_sequences(original_sentences), maxlen=max_seq_length, padding="post")

# Create target sequences for teacher forcing
target_input_sequences = target_sequences[:, :-1]
target_output_sequences = target_sequences[:, 1:]



In [5]:


embedding_dim = 256
latent_dim = 256
num_heads = 4

# Encoder
encoder_inputs = Input(shape=(max_seq_length,))
encoder_embedding = Embedding(vocab_size, embedding_dim)(encoder_inputs)
encoder_outputs = Dense(latent_dim)(encoder_embedding)

# Additional encoder layer
encoder_dense_2 = Dense(latent_dim, activation="relu")(encoder_outputs)
encoder_outputs = Add()([encoder_outputs, encoder_dense_2])  # Residual connection
encoder_outputs = LayerNormalization()(encoder_outputs)

# Decoder
decoder_inputs = Input(shape=(max_seq_length - 1,))
decoder_embedding = Embedding(vocab_size, embedding_dim)(decoder_inputs)

# Self-Attention
decoder_self_attention = MultiHeadAttention(num_heads=num_heads, key_dim=latent_dim)(decoder_embedding, decoder_embedding)
decoder_self_attention = Add()([decoder_embedding, decoder_self_attention])
decoder_self_attention = LayerNormalization()(decoder_self_attention)

# Cross-Attention
decoder_cross_attention = MultiHeadAttention(num_heads=num_heads, key_dim=latent_dim)(decoder_self_attention, encoder_outputs)
decoder_cross_attention = Add()([decoder_self_attention, decoder_cross_attention])
decoder_cross_attention = LayerNormalization()(decoder_cross_attention)

# Feed-forward layers
decoder_ffn = Dense(latent_dim * 4, activation="relu")(decoder_cross_attention)
decoder_ffn = Dense(latent_dim)(decoder_ffn)
decoder_ffn = Add()([decoder_cross_attention, decoder_ffn])  # Residual connection
decoder_ffn = LayerNormalization()(decoder_ffn)

# Additional feed-forward block
decoder_ffn_2 = Dense(latent_dim * 4, activation="relu")(decoder_ffn)
decoder_ffn_2 = Dense(latent_dim)(decoder_ffn_2)
decoder_ffn_2 = Add()([decoder_ffn, decoder_ffn_2])  # Residual connection
decoder_ffn_2 = LayerNormalization()(decoder_ffn_2)

# Output layer
decoder_outputs = Dense(vocab_size, activation="softmax")(decoder_ffn_2)

# Compile model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Prep train and test data -went with '42' for seed because it's the joke that keeps on giving
decoder_target_output = np.expand_dims(target_output_sequences, -1)
X_train, X_test, y_train, y_test, decoder_y_train, decoder_y_test = train_test_split(
    input_sequences, target_input_sequences, decoder_target_output, test_size=0.2, random_state=42
)


2024-12-05 19:11:54.558263: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [ ]:
# Train the decoder model 
print("Training the decoder model...")
model.fit(
    [X_train, y_train], decoder_y_train,
    validation_data=([X_test, y_test], decoder_y_test),
    batch_size=60,
    epochs=50,
)

Training the model...
Epoch 1/50


1/1 ━━━━━━━━━━━━━━━━━━━━ 27s 27s/step - accuracy: 0.5353 - loss: 3.2404 - val_accuracy: 0.4976 - val_loss: 3.8861
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 27s 27s/step - accuracy: 0.5368 - loss: 3.1796 - val_accuracy: 0.4999 - val_loss: 3.8801
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 27s 27s/step - accuracy: 0.5521 - loss: 3.1179 - val_accuracy: 0.4992 - val_loss: 3.8727
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 41s 41s/step - accuracy: 0.5582 - loss: 3.0593 - val_accuracy: 0.5001 - val_loss: 3.8715
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 27s 27s/step - accuracy: 0.5716 - loss: 3.0009 - val_accuracy: 0.5021 - val_loss: 3.8731
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 27s 27s/step - accuracy: 0.5804 - loss: 2.9439 - val_accuracy: 0.5020 - val_loss: 3.8735
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 27s 27s/step - accuracy: 0.5845 - loss: 2.8869 - val_accuracy: 0.5016 - val_loss: 3.8779
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 27s 27s/step - accuracy: 0.5892 - loss: 2.8338 - val_accuracy: 0.5017 - val_loss: 3.8822
Epoch 9/50


In [ ]:
# Save the model because it takes forever to fit the damn thing (i.e. 30min to fit an <1sec to load a saved model)
model.save("/home/ec2-user/kaggle/rudolph/model_v0.keras")

In [6]:
# Call saved model (note: keras file is too big to save on github so make sure to add in your .gitignore file)

model = load_model("/home/ec2-user/kaggle/rudolph/model_v0.keras")

In [7]:
# inference

encoder_model = Model(encoder_inputs, encoder_outputs)

# Decoder for inference
decoder_inputs_inf = Input(shape=(max_seq_length - 1,))
decoder_embedding_inf = Embedding(vocab_size, embedding_dim)(decoder_inputs_inf)

decoder_self_attention_inf = MultiHeadAttention(num_heads=4, key_dim=latent_dim)(decoder_embedding_inf, decoder_embedding_inf)
decoder_self_attention_inf = Add()([decoder_embedding_inf, decoder_self_attention_inf])
decoder_self_attention_inf = LayerNormalization()(decoder_self_attention_inf)

decoder_cross_attention_inf = MultiHeadAttention(num_heads=4, key_dim=latent_dim)(decoder_self_attention_inf, encoder_outputs)
decoder_cross_attention_inf = Add()([decoder_self_attention_inf, decoder_cross_attention_inf])
decoder_cross_attention_inf = LayerNormalization()(decoder_cross_attention_inf)

decoder_ffn_inf = Dense(latent_dim * 4, activation="relu")(decoder_cross_attention_inf)
decoder_ffn_inf = Dense(latent_dim)(decoder_ffn_inf)
decoder_ffn_inf = Add()([decoder_cross_attention_inf, decoder_ffn_inf])
decoder_ffn_inf = LayerNormalization()(decoder_ffn_inf)

decoder_outputs_inf = Dense(vocab_size, activation="softmax")(decoder_ffn_inf)

decoder_model = Model([decoder_inputs_inf, encoder_outputs], decoder_outputs_inf)

In [ ]:
# Hardest part of the project (for me at least) -  function to decode scrambled sentences
"""
Requirements:
1. ensuring the output is a permutation or original words
2. mask words that are not included in the original scrambled sentenc
3. mask words from the original scrambled sentence that are already used (unless they have multiple counts then count down until 0)
"""
"""
def decode_sequence(input_seq, scrambled_sentence):
 
    # Encode the input to get the context vector
    encoder_output = encoder_model.predict(input_seq)

    # Tokenize the scrambled sentence to get word counts
    scrambled_words = scrambled_sentence.split()
    word_count = Counter(scrambled_words)

    # Generate the initial target sequence with the start token
    target_seq = np.zeros((1, max_seq_length - 1))
    target_seq[0, 0] = tokenizer.word_index["<start>"]

    # Initialize variables for decoding
    stop_condition = False
    decoded_sentence = []
    used_words = set()

    while not stop_condition:
        # Predict the next token probabilities
        output_tokens = decoder_model.predict([target_seq, encoder_output])

        # Create a mask for valid words
        valid_word_indices = []
        for word, count in word_count.items():
            if count > 0:  # Only include words still available in scrambled input
                token_index = tokenizer.word_index.get(word, None)
                if token_index is not None:
                    valid_word_indices.append(token_index)

        # Mask out probabilities of words that are not in the scrambled set
        if valid_word_indices:
            mask = np.ones(output_tokens[0, -1, :].shape, dtype=bool)
            mask[valid_word_indices] = False  # Only allow valid word indices
            output_tokens[0, -1, mask] = 0
        else:
            # No valid words left, terminate decoding
            stop_condition = True
            continue

        # Resample words from the scambled set
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = tokenizer.index_word.get(sampled_token_index, None)

        # keep track of used words that haven't been over used, and add then to the used word list
        if sampled_word in word_count and word_count[sampled_word] > 0:
            decoded_sentence.append(sampled_word)
            word_count[sampled_word] -= 1
            used_words.add(sampled_word)

        # check progress 
        print(f"Resampled token index: {sampled_token_index}")
        print(f"Resampled word: {sampled_word}")

        # Check if decoding is complete
        if len(decoded_sentence) == len(scrambled_words) or sum(word_count.values()) == 0:
            stop_condition = True

        # Update the target sequence for the next iteration
        target_seq = np.zeros((1, max_seq_length - 1))
        target_seq[0, 0] = sampled_token_index

    # Validate that all words were used; if not, add missing words
    missing_words = [word for word in scrambled_words if word not in used_words]
    decoded_sentence.extend(missing_words)

    return " ".join(decoded_sentence).strip()

"""


In [ ]:
"""
def decode_sequence(input_seq, scrambled_sentence):
    
    Decode a scrambled sentence while ensuring the output is a permutation
    of the words in the scrambled sentence and avoiding consecutive repetitions.
    
    # Encode the input to get the context vector
    encoder_output = encoder_model.predict(input_seq)

    # Tokenize the scrambled sentence to get word counts
    scrambled_words = scrambled_sentence.split()
    word_count = Counter(scrambled_words)

    # Generate the initial target sequence with the start token
    target_seq = np.zeros((1, max_seq_length - 1))
    target_seq[0, 0] = tokenizer.word_index["<start>"]

    # Initialize variables for decoding
    stop_condition = False
    decoded_sentence = []
    used_words = set()

    while not stop_condition:
        # Predict the next token probabilities
        output_tokens = decoder_model.predict([target_seq, encoder_output])

        # Create a mask for valid words
        valid_word_indices = []
        for word, count in word_count.items():
            if count > 0:  # Only include words still available in scrambled input
                token_index = tokenizer.word_index.get(word, None)
                if token_index is not None:
                    valid_word_indices.append(token_index)

        # Mask out probabilities of words that are not in the scrambled set
        if valid_word_indices:
            mask = np.ones(output_tokens[0, -1, :].shape, dtype=bool)
            mask[valid_word_indices] = False  # Only allow valid word indices
            output_tokens[0, -1, mask] = 0
        else:
            # No valid words left, terminate decoding
            stop_condition = True
            continue

        # Resample a valid word, avoiding consecutive repetitions
        sorted_indices = output_tokens[0, -1, :].argsort()[::-1]  # Sort probabilities in descending order
        for token_index in sorted_indices:
            sampled_word = tokenizer.index_word.get(token_index, None)
            if (
                sampled_word in word_count and
                word_count[sampled_word] > 0 and
                (not decoded_sentence or sampled_word != decoded_sentence[-1])  # Avoid consecutive repetition
            ):
                break
        else:
            # If no valid word is found (edge case), terminate
            stop_condition = True
            continue

        # Add the sampled word to the decoded sentence
        decoded_sentence.append(sampled_word)
        word_count[sampled_word] -= 1
        used_words.add(sampled_word)

        # Debugging prints
        print(f"Resampled token index: {token_index}")
        print(f"Resampled word: {sampled_word}")

        # Check if decoding is complete
        if len(decoded_sentence) == len(scrambled_words) or sum(word_count.values()) == 0:
            stop_condition = True

        # Update the target sequence for the next iteration
        target_seq = np.zeros((1, max_seq_length - 1))
        target_seq[0, 0] = token_index

    # Validate that all words were used; if not, add missing words
    missing_words = [word for word in scrambled_words if word not in used_words]
    decoded_sentence.extend(missing_words)

    return " ".join(decoded_sentence).strip()
"""

In [8]:
#newest 
from collections import Counter
import numpy as np

def decode_sequence(input_seq, scrambled_sentence):
    """
    Decode a scrambled sentence while ensuring the output is a permutation
    of the words in the scrambled sentence and avoiding consecutive repetitions.
    """
    # Encode the input to get the context vector
    encoder_output = encoder_model.predict(input_seq)

    # Tokenize the scrambled sentence to get word counts
    scrambled_words = scrambled_sentence.split()
    word_count = Counter(scrambled_words)

    # Generate the initial target sequence with the start token
    target_seq = np.zeros((1, max_seq_length - 1))
    target_seq[0, 0] = tokenizer.word_index["<start>"]

    # Initialize variables for decoding
    stop_condition = False
    decoded_sentence = []

    while not stop_condition:
        # Predict the next token probabilities
        output_tokens = decoder_model.predict([target_seq, encoder_output])

        # Create a mask for valid words
        valid_word_indices = []
        for word, count in word_count.items():
            if count > 0:  # Only include words still available in scrambled input
                token_index = tokenizer.word_index.get(word, None)
                if token_index is not None:
                    valid_word_indices.append(token_index)

        # Mask out probabilities of words that are not in the scrambled set
        if valid_word_indices:
            mask = np.ones(output_tokens[0, -1, :].shape, dtype=bool)
            mask[valid_word_indices] = False  # Only allow valid word indices
            output_tokens[0, -1, mask] = 0
        else:
            # No valid words left, terminate decoding
            stop_condition = True
            continue

        # Resample a valid word, avoiding consecutive repetitions
        sorted_indices = output_tokens[0, -1, :].argsort()[::-1]  # Sort probabilities in descending order
        sampled_word = None
        for token_index in sorted_indices:
            word_candidate = tokenizer.index_word.get(token_index, None)
            if (
                word_candidate in word_count and
                word_count[word_candidate] > 0 and
                (not decoded_sentence or word_candidate != decoded_sentence[-1])  # Avoid consecutive repetition
            ):
                sampled_word = word_candidate
                break

        # If no valid word is found, terminate
        if sampled_word is None:
            stop_condition = True
            continue

        # Add the sampled word to the decoded sentence
        decoded_sentence.append(sampled_word)
        word_count[sampled_word] -= 1

        # Debugging prints
        print(f"Resampled token index: {token_index}")
        print(f"Resampled word: {sampled_word}")

        # Check if decoding is complete
        if sum(word_count.values()) == 0:
            stop_condition = True

        # Update the target sequence for the next iteration
        target_seq = np.zeros((1, max_seq_length - 1))
        target_seq[0, 0] = token_index

    # Ensure all words are included
    missing_words = []
    for word, count in word_count.items():
        if count > 0:
            missing_words.extend([word] * count)
    decoded_sentence.extend(missing_words)

    return " ".join(decoded_sentence).strip()


In [10]:
# toss together some tests
scrambled_input = "row your boat down the stream gently row row"
#scrambled_input ="on the sat mat cat the"
#scrambled_input ="are ? you where"

input_sequence = pad_sequences(tokenizer.texts_to_sequences([scrambled_input]), maxlen=max_seq_length, padding="post")
unscrambled_sentence = decode_sequence(input_sequence, scrambled_input)
print(f"Scrambled: {scrambled_input}")
print(f"Unscrambled: {unscrambled_sentence}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step


2024-12-05 14:57:51.799355: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1408951296 exceeds 10% of free system memory.
2024-12-05 14:57:54.234182: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1409101440 exceeds 10% of free system memory.


1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
Resampled token index: 984
Resampled word: row


2024-12-05 14:57:57.248273: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1408951296 exceeds 10% of free system memory.
2024-12-05 14:57:59.659241: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1409101440 exceeds 10% of free system memory.


1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
Resampled token index: 61
Resampled word: your


2024-12-05 14:58:02.682114: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1408951296 exceeds 10% of free system memory.


1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
Resampled token index: 984
Resampled word: row
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
Resampled token index: 978
Resampled word: gently
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
Resampled token index: 984
Resampled word: row
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
Resampled token index: 46
Resampled word: down
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
Resampled token index: 1
Resampled word: the
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
Resampled token index: 3853
Resampled word: boat
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
Scrambled: row your boat down the stream gently row row
Unscrambled: row your row gently row down the boat stream


In [13]:
text_0 = "advent chimney elf family fireplace gingerbread mistletoe ornament reindeer scrooge"
text_1 = "advent chimney elf family fireplace gingerbread mistletoe ornament reindeer scrooge walk give jump drive bake the sleep night laugh and"
text_2 = "yuletide decorations gifts cheer holiday carol magi nutcracker polar grinch sleigh chimney workshop stocking ornament holly jingle beard naughty nice"
text_3 = "yuletide decorations gifts cheer holiday carol magi nutcracker polar grinch sleigh chimney workshop stocking ornament holly jingle beard naughty nice sing cheer and of the is eat visit relax unwrap"
text_4 = "hohoho candle poinsettia snowglobe peppermint eggnog fruitcake chocolate candy puzzle game doll toy workshop wonder believe dream hope peace joy merry season greeting card wrapping paper bow fireplace night cookie milk star wish wreath angel the to of and in that have it not with as you from we kaggle"
text_5 = "advent chimney elf family fireplace gingerbread mistletoe ornament reindeer scrooge walk give jump drive bake the sleep night laugh and yuletide decorations gifts cheer holiday carol magi nutcracker polar grinch sleigh chimney workshop stocking ornament holly jingle beard naughty nice sing cheer and of the is eat visit relax unwrap hohoho candle poinsettia snowglobe peppermint eggnog fruitcake chocolate candy puzzle game doll toy workshop wonder believe dream hope peace joy merry season greeting card wrapping paper bow fireplace night cookie milk star wish wreath angel the to of and in that have it not with as you from we kaggle"
scrambled_input = text_1
input_sequence = pad_sequences(tokenizer.texts_to_sequences([scrambled_input]), maxlen=max_seq_length, padding="post")
unscrambled_sentence = decode_sequence(input_sequence, scrambled_input)
print(f"Scrambled: {scrambled_input}")
print(f"Unscrambled: {unscrambled_sentence}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 463ms/step
Resampled token index: 4277
Resampled word: reindeer
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 429ms/step
Resampled token index: 821
Resampled word: walk
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 443ms/step
Resampled token index: 226
Resampled word: night
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 439ms/step
Resampled token index: 669
Resampled word: sleep
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 437ms/step
Resampled token index: 878
Resampled word: laugh
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 433ms/step
Resampled token index: 154
Resampled word: give
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 435ms/step
Resampled token index: 3862
Resampled word: ornament
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 439ms/step
Resampled token index: 1842
Resampled word: family
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 435ms/step
Resampled token index: 1
Resampled word: the
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 429ms/step
Resampled token index: 2
Resampled word: and
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 427ms/step
Scrambled: advent chimney elf family firepl

In [9]:
csv_file = "/home/ec2-user/kaggle/rudolph/sample_submission.csv"
input = pd.read_csv(csv_file)
input = input.drop(columns=['id'])


In [38]:

# Loop through each row in column 'XX'
for index, value in input['text'].items():
    # Create a variable for the string in column 'XX'
    current_string = value
    print(f"Row {index}: {current_string}")

Row 0: advent chimney elf family fireplace gingerbread mistletoe ornament reindeer scrooge
Row 1: advent chimney elf family fireplace gingerbread mistletoe ornament reindeer scrooge walk give jump drive bake the sleep night laugh and
Row 2: yuletide decorations gifts cheer holiday carol magi nutcracker polar grinch sleigh chimney workshop stocking ornament holly jingle beard naughty nice
Row 3: yuletide decorations gifts cheer holiday carol magi nutcracker polar grinch sleigh chimney workshop stocking ornament holly jingle beard naughty nice sing cheer and of the is eat visit relax unwrap
Row 4: hohoho candle poinsettia snowglobe peppermint eggnog fruitcake chocolate candy puzzle game doll toy workshop wonder believe dream hope peace joy merry season greeting card wrapping paper bow fireplace night cookie milk star wish wreath angel the to of and in that have it not with as you from we kaggle
Row 5: advent chimney elf family fireplace gingerbread mistletoe ornament reindeer scrooge wal

In [10]:
output_file = []
for index, value in input['text'].items():
    # Create a variable for the string in column 'XX'
    scrambled_input = value
    #print(f"Row {index}: {current_string}")

    input_sequence = pad_sequences(tokenizer.texts_to_sequences([scrambled_input]), maxlen=max_seq_length, padding="post")
    unscrambled_sentence = decode_sequence(input_sequence, scrambled_input)
    output_file.append(unscrambled_sentence)
    print(f"Scrambled: {scrambled_input}")
    print(f"Unscrambled: {unscrambled_sentence}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 16s 16s/step
Resampled token index: 4806
Resampled word: reindeer
1/1 ━━━━━━━━━━━━━━━━━━━━ 15s 15s/step
Resampled token index: 1547
Resampled word: family
1/1 ━━━━━━━━━━━━━━━━━━━━ 15s 15s/step
Resampled token index: 4638
Resampled word: ornament
1/1 ━━━━━━━━━━━━━━━━━━━━ 15s 15s/step
Scrambled: advent chimney elf family fireplace gingerbread mistletoe ornament reindeer scrooge
Unscrambled: reindeer family ornament advent chimney elf fireplace gingerbread mistletoe scrooge
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 15s 15s/step
Resampled token index: 202
Resampled word: night
1/1 ━━━━━━━━━━━━━━━━━━━━ 15s 15s/step
Resampled token index: 417
Resampled word: sleep
1/1 ━━━━━━━━━━━━━━━━━━━━ 15s 15s/step
Resampled token index: 4806
Resampled word: reindeer
1/1 ━━━━━━━━━━━━━━━━━━━━ 15s 15s/step
Resampled token index: 3
Resampled word: and
1/1 ━━━━━━━━━━━━━━━━━━━━ 15s 15s/step
Resampled token index: 1
Resampled w

In [11]:
unscrambled = pd.DataFrame(output_file, columns=["text"])

# Display the DataFrame
#print(input)
#print(unscrambled)
#print(output_file)
#print(max_seq_length)
 # Check that each submitted string is a permutation of the solution string
sol_counts = unscrambled.loc[:, 'text'].str.split().apply(Counter)

print(f"solution counts = {sol_counts}")
sub_counts = input.loc[:, 'text'].str.split().apply(Counter)

print(f"input counts = {sub_counts}")
invalid_mask = sol_counts != sub_counts

if invalid_mask.any():
    raise ParticipantVisibleError(
        'At least one submitted string is not a valid permutation of the solution string.'
    )

print(invalid_mask)


solution counts = 0    {'reindeer': 1, 'family': 1, 'ornament': 1, 'a...
1    {'night': 1, 'sleep': 1, 'reindeer': 1, 'and':...
2    {'ornament': 1, 'nice': 1, 'yuletide': 1, 'dec...
3    {'and': 1, 'the': 1, 'visit': 1, 'is': 1, 'orn...
4    {'it': 1, 'night': 1, 'as': 1, 'peace': 1, 'in...
5    {'it': 1, 'night': 2, 'as': 1, 'peace': 1, 'sl...
Name: text, dtype: object
input counts = 0    {'advent': 1, 'chimney': 1, 'elf': 1, 'family'...
1    {'advent': 1, 'chimney': 1, 'elf': 1, 'family'...
2    {'yuletide': 1, 'decorations': 1, 'gifts': 1, ...
3    {'yuletide': 1, 'decorations': 1, 'gifts': 1, ...
4    {'hohoho': 1, 'candle': 1, 'poinsettia': 1, 's...
5    {'advent': 1, 'chimney': 2, 'elf': 1, 'family'...
Name: text, dtype: object
0    False
1    False
2    False
3    False
4    False
5    False
Name: text, dtype: bool


In [54]:
print(invalid_mask)

0    False
1    False
2    False
3    False
4    False
5    False
Name: text, dtype: bool


In [27]:
# Calculate Perpexity using kaggle's competition script 
%run '/home/ec2-user/kaggle/rudolph/perplexityCalc.py'

In [28]:

model_path = "/home/ec2-user/.cache/kagglehub/models/google/gemma-2/transformers/gemma-2-9b/2"
score(unscrambled, input, 'text', model_path=model_path, clear_mem=True) > 0

Loading checkpoint shards: 100%|██████████| 8/8 [00:02<00:00,  3.58it/s]


True

In [29]:
scores = score(unscrambled, input, 'text', model_path=model_path, clear_mem=True)

Loading checkpoint shards: 100%|██████████| 8/8 [00:01<00:00,  4.46it/s]


In [ ]:
print(scores)
# Score1 = 2174.8712132759915

2174.8712132759915


In [31]:
perplex = PerplexityCalculator(model_path=model_path)

Loading checkpoint shards: 100%|██████████| 8/8 [00:01<00:00,  4.38it/s]


In [32]:
perplex.get_perplexity(unscrambled["text"].tolist())

[2008.0405307054234,
 23470.653356597497,
 1944.3116912824626,
 3400.713597052891,
 1385.518197259851,
 716.6361653971617]

In [33]:
perplex.get_perplexity(input["text"].tolist())

[3877.722785267886,
 6059.756959306958,
 1116.90819498697,
 1286.9573745451812,
 352.73848538785325,
 355.14348016109784]